<style>
    h1 {
        padding: 8px 8px;
        background-image: linear-gradient(135deg, #c9f3ff, rgb(131, 218, 255));
        font-weight: 700;
        position: static;
        text-align: center;
        color: #006098;
        font-size: 36px;
    }
    h2 {
        font-weight: 700;
        text-align: center;
        font-style: italic;
        font-size: 24px;
    }
</style>

<div><h1>XÂY DỰNG MODEL</h1></div>
<div><h2>SVM model via SVR</h2></div>

# Khai báo thư viện cần thiết

In [44]:
import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
# ! pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
! pip install pyspellchecker
from spellchecker import SpellChecker
# ! pip install spacy
from spacy import load
from urllib.parse import urlparse

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVR
from sklearn.metrics import cohen_kappa_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dagngyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 140.0 kB/s eta 0:00:0000:0100:02


# Đọc dữ liệu

- Đọc dữ liệu trên Google Drive.

In [2]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [3]:
# train = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/train.csv')
# train.sample(5)

In [4]:
# test = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/test.csv')
# test

- Đọc dữ liệu trên local path.

In [5]:
train = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/train.csv')
train.sample(5)

,essay_id,full_text,score
9016,8525a80,"Dear senator,\n\nI am writing to you from the ...",3
12327,b595104,"To Whom It May Concern,\n\nThe Electoral Colle...",4
14212,d129f6b,"Sure, driverless cars sound interesting, but a...",4
1401,15ec1e4,Why We Should Study Venus\n\nWhat is venus lik...,4
7461,6fd79d3,"The author in ""Driverless Cars Are Coming"" pre...",4


In [6]:
test = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/test.csv')
test.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [7]:
# Remove score from train and concate with test
all_data = pd.concat([train.iloc[:,:-1], test], axis=0).reset_index(drop=True)
all_data.sample(5)

,essay_id,full_text
7829,7570efd,Do you believe in aliens or life on other plan...
5780,56ae3e1,"The Challenge of Exploring Venus, a article ab..."
3359,31e00b5,"""With less cars we have less accidents and les..."
7436,6f7ae5d,I do not agree with the idea of completey driv...
16692,f60205d,The Facial Action Coding System created by Dr....


# Xử lý dữ liệu

Ta sẽ loại bỏ các ký tự dư thừa và chuỗi không cung cấp nhiều ý nghĩa thông tin.

In [8]:
def remove_excess_str(text):
    # Loại bỏ các username bắt đầu @
    text = re.sub("@\w+", '', text)
    # Loại bỏ các thẻ HTML
    html = re.compile(r'<.*?>')
    text = html.sub(r'', text)
    # Loại bỏ URL
    text = [word for word in text.split() if not urlparse(word).scheme]
    text = ' '.join(text)
    # Loại bỏ dấu nháy đơn mà theo sau nó là chữ số
    text = re.sub("'\d+", "", text)
    # Loại bỏ các ký tự dư thừa như khoảng trắng, dấu chấm, dấu phẩy
    text = re.sub(r"\.+", ".", text)
    text = re.sub(r"\,+", ",", text)
    text = ' '.join(text.split())
    return text

# Trích lọc đặc trưng từ dữ liệu

Đầu tiên, ta định nghĩa và cài đặt hàm đếm số lượng từ vựng bị sử dụng sai.

In [45]:
def count_spelling_errors(word_list):
    return len(list(SpellChecker().unknown(word_list)))

Tiếp theo, định nghĩa hàm loại bỏ dấu câu để thống kê thông số về từ.

In [10]:
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

Tiếp theo, cài đặt hàm phân tách thành các đoạn văn, thành các câu để thống kê.

In [11]:
def split_paragraphs(text):
    new_text = re.split('\n\n', text)
    return new_text

Cài đặt hàm trích lọc những từ khóa đóng góp quan trọng trong ý nghĩa của câu.

In [12]:
# Load Spacy model
nlp = load("en_core_web_sm")

# Define stop words
stop_words = set(stopwords.words('english'))

# Preprocess the text
def filter_word(text):
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word.isalnum() and word not in stop_words]
    return words

Phần này, ta sẽ quan sát những giá trị thống kê từ văn bản:
- Tổng số lượng đoạn văn;
- Thống kê theo đoạn văn: Sau khi phân tách văn bản và xử lý dữ liệu, ta sẽ thống kê:
    + Số lượng câu: Giá trị nhỏ nhất, tứ phân vị thứ nhất, trung vị, tứ phân vị thứ ba, giá trị lớn nhất;
    + Số lượng từ: Giá trị nhỏ nhất, tứ phân vị thứ nhất, trung vị, tứ phân vị thứ ba, giá trị lớn nhất;
- Tổng số lượng câu;
- Thống kê theo câu: Sau khi phân tách thành các câu và xử lý dữ liệu, ta sẽ thống kê:
    + Số lượng từ: Giá trị nhỏ nhất, tứ phân vị thứ nhất, trung vị, tứ phân vị thứ ba, giá trị lớn nhất;
- Tổng số lượng từ;
- Tổng số lượng từ sau khi trích lọc những từ đóng góp ý nghĩa cho câu, đoạn văn;
- Thống kê theo từ: Trích lọc những từ đóng góp ý nghĩa cho câu, đoạn văn:
    + Số lượng ký tự: Giá trị nhỏ nhất, tứ phân vị thứ nhất, trung vị, tứ phân vị thứ ba, giá trị lớn nhất;
- Số lượng từ loại sai chính tả trong đoạn văn.

In [14]:
def paragraph_engineering(paragraph):
    _paragraph = list(map(remove_excess_str, paragraph))
    _lst_sents = []
    _lst_words = []
    for para in _paragraph:
        _lst_sents.append(len(sent_tokenize(para)))
        _lst_words.append(len(word_tokenize(para)))
    return len(_paragraph),\
            np.min(_lst_sents), np.quantile(_lst_sents, 0.25), np.median(_lst_sents), np.quantile(_lst_sents, 0.75), np.max(_lst_sents),\
            np.min(_lst_words), np.quantile(_lst_words, 0.25), np.median(_lst_words), np.quantile(_lst_words, 0.75), np.max(_lst_words)

In [15]:
all_data['num_para'],\
    all_data['para_min_sent'], all_data['para_q1_sent'], all_data['para_median_sent'], all_data['para_q3_sent'], all_data['para_max_sent'],\
    all_data['para_min_word'], all_data['para_q1_word'], all_data['para_median_word'], all_data['para_q3_word'], all_data['para_max_word'] = zip(*all_data['full_text'].map(split_paragraphs).map(paragraph_engineering))

In [16]:
def sentence_engineering(sentence):
    _lst_words = []
    for sent in sentence:
        _lst_words.append(len(word_tokenize(sent)))
    return len(sentence),\
            np.min(_lst_words), np.quantile(_lst_words, 0.25), np.median(_lst_words), np.quantile(_lst_words, 0.75), np.max(_lst_words)

In [17]:
all_data['num_sent'],\
    all_data['sent_min_word'], all_data['sent_q1_word'], all_data['sent_median_word'], all_data['sent_q3_word'], all_data['sent_max_word']\
        = zip(*all_data['full_text'].apply(remove_excess_str).map(sent_tokenize).map(sentence_engineering))

In [18]:
def word_engineering(word):
    _lst_chars = list(map(len, word))
    return len(word), np.min(_lst_chars), np.quantile(_lst_chars, 0.25), np.median(_lst_chars), np.quantile(_lst_chars, 0.75), np.max(_lst_chars)

In [25]:
word_full_text = all_data['full_text'].map(remove_excess_str).map(remove_punctuation)
word_full_text_no_stopwords = word_full_text.map(filter_word)
# all_data['num_error'] = word_full_text.map(word_tokenize).map(count_spelling_errors)
all_data['num_word'] = word_full_text.map(word_tokenize).map(len)
all_data['num_word_no_stopwords'], all_data['word_min_chars'], all_data['word_q1_chars'], all_data['word_median_chars'], all_data['word_q3_chars'], all_data['word_max_chars'] = zip(*word_full_text_no_stopwords.map(word_engineering))

Số lượng đặc trung sau khi rút trích thông tin từ dữ liệu.

In [52]:
all_data.shape[1] - 2

(17310, 26)

# Trích lọc đặc trưng thông qua TF-IDF

"TF-IDF là viết tắt của “Term Frequency, Inverse Document Frequency” - tạm dịch “Tần suất thuật ngữ, Tần suất tài liệu nghịch đảo”. Đó là một cách để chấm điểm tầm quan trọng của các từ (hoặc \"các thuật ngữ\") dựa trên tần suất xuất hiện của chúng xuất hiện trên nhiều tài liệu dựa trên quy tắc sau:"
- Nếu một từ xuất hiện thường xuyên trong tài liệu, điều đó rất quan trọng. Cho từ này điểm cao.
- Nhưng nếu một từ xuất hiện trong nhiều tài liệu, thì đó không phải là mã định danh duy nhất. Cho từ đó điểm thấp.

Do đó, những từ phổ biến như `the` và `for` xuất hiện trong nhiều tài liệu sẽ được scaled down. Các từ xuất hiện thường xuyên trong một tài liệu sẽ được scaled up.

Với những giải thích trên, ta có công thức tính trọng số của một từ trong tài liệu trong ngữ liệu như sau:
$$w_{i,j} = tf_{i,j} \cdot idf_i = tf_{i,j} \cdot log(\frac {N}{df_i})$$

Trong đó:
- $tf_{i,j}$: Tần suất xuất hiện của i trong j
- $N$: Tổng số tài liệu
- $df_i$: Số tài liệu chứa i

**Reference:** https://medium.com/analytics-vidhya/an-introduction-to-tf-idf-using-python-5f9d1a343f77"

In [58]:
# Khởi tạo TfidfVectorizer với các tham số cụ thể
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 3),
    min_df=0.05,
    max_df=0.95,
    sublinear_tf=True,
)

# Sử dụng TfidfVectorizer cho dữ liệu
tfidf = vectorizer.fit_transform([row for row in all_data['full_text']]).toarray()

# Lưu kết quả
colums_tfidf = [f'tfidf_{i}' for i in range(tfidf.shape[1])]
tfidf_df = pd.DataFrame(tfidf, columns=colums_tfidf)
tfidf_df['essay_id'] = all_data['essay_id']
all_data = all_data.merge(tfidf_df, on='essay_id', how='left')

In [59]:
all_data

,essay_id,full_text,num_para,para_min_sent,para_q1_sent,para_median_sent,para_q3_sent,para_max_sent,para_min_word,para_q1_word,...,tfidf_3282,tfidf_3283,tfidf_3284,tfidf_3285,tfidf_3286,tfidf_3287,tfidf_3288,tfidf_3289,tfidf_3290,tfidf_3291
0,000d118,Many people have car where they live. The thin...,1,13,13.0,13.0,13.0,13,545,545.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.034733,0.071065,0.0,0.0
1,000d118,Many people have car where they live. The thin...,1,13,13.0,13.0,13.0,13,545,545.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.034733,0.071065,0.0,0.0
2,000fe60,I am a scientist at NASA that is discussing th...,5,2,3.0,3.0,5.0,8,44,52.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,000fe60,I am a scientist at NASA that is discussing th...,5,2,3.0,3.0,5.0,8,44,52.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,001ab80,People always wish they had the same technolog...,4,4,4.0,5.5,7.5,9,92,104.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17311,000d118,Many people have car where they live. The thin...,1,13,13.0,13.0,13.0,13,545,545.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.034733,0.071065,0.0,0.0
17312,000fe60,I am a scientist at NASA that is discussing th...,5,2,3.0,3.0,5.0,8,44,52.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
17313,000fe60,I am a scientist at NASA that is discussing th...,5,2,3.0,3.0,5.0,8,44,52.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
17314,001ab80,People always wish they had the same technolog...,4,4,4.0,5.5,7.5,9,92,104.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
